In [ ]:
!mamba install -y -c bioconda pysradb
!mkdir assembly_info.biosample.accession.output

In [ ]:
import sys
import time
import pandas as pd

from pysradb import SRAweb

db = SRAweb()
results=list()

file = open('SRR.txt',"r")
for srp in file:
    try:
        df = db.sra_metadata(srp)
        #df.to_csv("assembly_info.biosample.accession.output/{}.tsv".format(srp), sep="\t", index=False)
        results.append(df)
    except:
        sys.stderr.write("Error with {}\n".format(srp))
        time.sleep(0.5)
    time.sleep(0.5)

In [ ]:
final_df=pd.concat(results, axis=0, ignore_index=False)
final_df.to_csv('assembly_info.biosample.accession.output.tsv',sep="\t")